In [0]:
import numpy as np
import pandas as pd
import cv2
import os
import glob
import sys
import itertools
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.segmentation import active_contour


In [0]:
class IMG_list:
    def __init__(self, path):
        self.path = path


    def List(path):
        img_list=[]
        img_paths=[]
        for r,d, f in os.walk(path):
            for names in d:
                if names.endswith('img'):
                    for root, dir, files in os.walk(names):
                        if dir !=[]:
                            os.chdir(os.path.join(str(path),str(names), str(dir[0])))
                            path=os.getcwd()             
                            for r,d,f in os.walk(path):
                                for files in f:
                                    if files.endswith('.bmp'):
                                        img_list.append(os.path.join(str(r),str(files)))
        return (img_list)


In [0]:
class Preprocessing:
    def PrePro(img):
        cv2.imshow('immagine',img)
        cv2.waitKey(0)  
        cv2.destroyAllWindows()
        image = img.copy()
        # blur image
        image = cv2.GaussianBlur(image, (5, 5), 0)
        # Applying CLAHE to resolve uneven illumination
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])
        image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        kernel = np.ones((11, 11), np.uint8)
        for i in range(image.shape[-1]):
            image[:, :, i] = cv2.morphologyEx(
            image[:, :, i],
            cv2.MORPH_CLOSE, kernel)
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        cv2.imshow('hsv', hsv_image)
        cv2.waitKey(0)
        return (hsv_image)
                
    def Lines(img,Open):            #Funzione per cercare di disegnare i contorni del neo
        

        new_img=img.copy()
        gray= cv2.cvtColor(new_img, cv2.COLOR_RGB2GRAY)
        for i in range (0,5):
            new_img = cv2.medianBlur(gray, 27)
        cv2.imshow('Blurred', new_img)
        cv2.waitKey(0)
        ret2,thresh2 = cv2.threshold(new_img,127,255,cv2.THRESH_BINARY_INV)
        ret, thresh = cv2.threshold(thresh2, 170, 180, 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
        
        cv2.drawContours(new_img, contours,-1,(0,255,253),thickness=3)
        cv2.imshow('Snake', new_img)
        cv2.waitKey(0)
        # Isolate largest contour
        contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
        biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]
      
        mask = np.zeros(img.shape, np.uint8)

        cv2.drawContours(mask, [biggest_contour], -1, 255, -1)

        cv2.waitKey(0)
        cv2.imshow('mask', mask)
        cv2.waitKey(0)
     

In [0]:
path=os.getcwd()
mystring=IMG_list.List(path)
processing=Preprocessing()
iterations=3
for names in mystring:
    img=cv2.imread(names)
    Open=Preprocessing.PrePro(img)
    Preprocessing.Lines(img, Open)
    #centro=Preprocessing.Center(img)
    #for u in range(len(centro[0])):
    #    punti=Preprocessing.Cerchio(1000,centro[0][u],centro[1])[:-1]
     #   snake = active_contour(img, punti)
      #  cv2.polylines(img, np.int32([snake]), True, (0,255,255), 3)
       # cv2.imshow('Snake', img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()